# Simple Hardware Test
A few lines of Python to check a possible hardware acceleration on your machine.

Companion programs of the book: [_Python for Natural Language Processing_](https://link.springer.com/book/9783031575488)

__Author__: Pierre Nugues

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm

## Selecting the Device

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
device

device(type='mps')

## Matrix Multiplication

In [4]:
SIZE = 1000
ITER = 50000
BATCH_SIZE = 32

### CPU

In [5]:
M = torch.rand(SIZE, SIZE)
N = torch.rand(SIZE, SIZE)

In [6]:
for _ in tqdm(range(ITER)):
    M @ N

100%|██████████| 50000/50000 [01:09<00:00, 720.09it/s]


### GPU

In [7]:
M_dev = torch.rand(SIZE, SIZE).to(device)
N_dev = torch.rand(SIZE, SIZE).to(device)

In [8]:
for _ in tqdm(range(ITER)):
    M_dev @ N_dev

100%|██████████| 50000/50000 [00:20<00:00, 2432.66it/s]


## Models

In [9]:
model = nn.Sequential(nn.Linear(SIZE, SIZE),
                      nn.ReLU(),
                      nn.Linear(SIZE, SIZE))

### CPU

In [10]:
x = torch.rand(SIZE)

In [11]:
for _ in tqdm(range(ITER)):
    model(x)

100%|██████████| 50000/50000 [00:02<00:00, 20239.07it/s]


### GPU

In [12]:
x_dev = x.to(device)

In [13]:
model_dev = model.to(device)

In [14]:
for _ in tqdm(range(ITER)):
    model_dev(x_dev)

100%|██████████| 50000/50000 [00:03<00:00, 14145.85it/s]


## Batched Inputs

In [15]:
model = nn.Sequential(nn.Linear(SIZE, SIZE),
                      nn.ReLU(),
                      nn.Linear(SIZE, SIZE))

### CPU

In [16]:
X = torch.rand(BATCH_SIZE, SIZE)

In [17]:
for _ in tqdm(range(ITER)):
    model(X)

100%|██████████| 50000/50000 [00:17<00:00, 2783.00it/s]


### GPU

In [18]:
X_dev = X.to(device)

In [19]:
for _ in tqdm(range(ITER)):
    model_dev(X_dev)

100%|██████████| 50000/50000 [00:04<00:00, 12106.66it/s]


## Compiled Models

In [20]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [21]:
model = nn.Sequential(nn.Linear(SIZE, SIZE),
                      nn.ReLU(),
                      nn.Linear(SIZE, SIZE))

In [22]:
model_compiled = torch.compile(model)

### CPU

In [23]:
for _ in tqdm(range(ITER)):
    model_compiled(X)

100%|██████████| 50000/50000 [00:22<00:00, 2188.83it/s]


### GPU

In [24]:
model_compiled = torch.compile(model)

In [25]:
X_dev = X.to(device)

In [26]:
model_compiled_dev = model_compiled.to(device)

You have to run this loop twice on macOS

In [28]:
for _ in tqdm(range(ITER)):
    model_compiled_dev(X_dev)

100%|██████████| 50000/50000 [00:04<00:00, 11762.04it/s]
